In [172]:
import polars as pl
import numpy as np
import requests
from zipfile import ZipFile
import os 

import psutil

In [105]:


# Get total memory
total_memory = psutil.virtual_memory().total / (1024 ** 3)  # Convert to GB
available_memory = psutil.virtual_memory().available / (1024 ** 3)  # Convert to GB

print(f"Total memory: {total_memory:.2f} GB")
print(f"Available memory: {available_memory:.2f} GB")

Total memory: 1007.09 GB
Available memory: 972.97 GB


In [106]:
# Download file anew: (takes 13 m 12s)
chunk_size = 4096
filename = "C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/Xena/TCGA_PANCAN_transcript_expression_RNAseq_RSEM_tpm.gz"
document_url = "https://toil-xena-hub.s3.us-east-1.amazonaws.com/download/tcga_rsem_isoform_tpm.gz"
#with requests.get(document_url, stream=True) as r:
#        with open(filename, 'wb') as f:
#            for chunk in r.iter_content(chunk_size): 
#                if chunk:
#                    f.write(chunk)

In [167]:
in_file = "/zhome/94/f/147417/tma_thesis/data/xena/TCGA_PANCAN_transcript_expression_RNAseq_RSEM_tpm.tsv" #"C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/Xena/TCGA_PANCAN_transcript_expression_RNAseq_RSEM_tpm.gz"
in_pheno2 = "/zhome/94/f/147417/tma_thesis/data/xena/TCGA_PANCAN_curated_clinical_data.tsv" # "C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/Xena/TCGA_PANCAN_curated_clinical_data.tsv"
#in_transcript_id = "/zhome/94/f/147417/tma_thesis/data/xena/TCGA_PANCAN_transcript_expression_RNAseq_RSEM_transcript_list.txt" # "C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/Xena/TCGA_PANCAN_transcript_expression_RNAseq_RSEM_transcript_list.txt"
in_genenames = "/zhome/94/f/147417/tma_thesis/data/targets_ensg.csv" # "C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/targets_ensg.csv"

In [168]:
# Load dataframe as lazy using polars
lf_data = pl.scan_csv(in_file, separator="\t")

# Load transcript ids:
#with open(in_transcript_id) as f:  
#    transcript_id_list = f.read().splitlines()
#transcript_id_list.pop(0) # Remove "column name"s

# Add it to the lf_data: 
#lf_data = lf_data.with_columns(trans_id = transcript_id_list)

# Remove trailing . and digits on the transcript ids:
lf_data = lf_data.with_columns(pl.col("sample").str.split(".").list.get(0))

# Load gene names as dataframe: 
df_genenames = pl.read_csv(in_genenames)

In [169]:
# Define a function to fetch transcript IDs from Ensembl
def fetch_transcript_ids_from_ensembl(gene_id):
    url = f"https://rest.ensembl.org/lookup/id/{gene_id}?expand=1"
    response = requests.get(url, headers={"Content-Type": "application/json"})
    if response.ok:
        data = response.json()
        return [transcript['id'] for transcript in data.get('Transcript', [])]
    else:
        return []
    
# Define a function to fetch transcript IDs from Ensembl
def fetch_gene_ids_from_ensembl(trans_id):
    url = f"https://rest.ensembl.org/lookup/id/{trans_id}?expand=1"
    response = requests.get(url, headers={"Content-Type": "application/json"})
    if response.ok:
        data = response.json()
        return data.get('display_name', [])
    else:
        return []

In [178]:
# From ensembl API get the transcript IDs for all genes of interest: 
# Apply the API function to the DataFrame
df_genenames = df_genenames.with_columns(pl.col("converted_alias").map_elements(fetch_transcript_ids_from_ensembl).alias("transcript_ids"))

# Get a list of all the transcript IDs related to any of the 14 genes of interest: 
trans_id_of_interest = df_genenames.select(pl.col("transcript_ids").explode()).to_series().to_list()

def make_cancer_specific_tsv(cancer_type_abb, write_file):
    # Make list of sample ids for different cancer types
    df_pheno = pl.read_csv(in_pheno2, separator="\t")
    sample_id_cancertype = df_pheno.filter(pl.col("cancer type abbreviation") == cancer_type_abb).select(pl.col("sample")).to_numpy()
    #sample_id_data = np.array(lf_data.collect_schema().names())
    sample_id_data = np.array(lf_data.limit(1).collect().columns)
    sample_id_cancertype = set(np.intersect1d(sample_id_cancertype, sample_id_data))

    # Subset lazyframe to include only samples from cancer type of interest.
    lf_data_cancertype = lf_data.select(sample_id_cancertype.union(set(["sample"])))

    # Use list to filter the data: 
    lf_data_cancertype = lf_data_cancertype.filter(pl.col("sample").is_in(trans_id_of_interest))

    ### ##### ##### 
    #### FIX THIS ISSUE SO YOU CAN ADD GENE ID AND SAVE NEW TAB DELIM. FILES !!!
    # Add column with gene_id: 
    #lf_data_cancertype = lf_data_cancertype.with_columns(pl.col("sample").map_elements(fetch_gene_ids_from_ensembl).alias("gene_id"))

    # Put rename "sample" columns and move to first posititon.
    all_columns = lf_data_cancertype.columns
    #lf_data_cancertype = lf_data_cancertype.rename(
    #    {"sample": "transcript_id"}
    #).select(
    #    [pl.col("transcript_id"), pl.col("gene_id")]  + [pl.col(c) for c in all_columns if c not in ["gene_id", "transcript_id"]] 
    #)

    out_path = "/zhome/94/f/147417/tma_thesis/data/xena/TCGA_PANCAN_transcript_expression_RNAseq_RSEM_tpm_{}.tsv".format(cancer_type_abb)

    if write_file: 
        if os.path.exists(out_path):
            os.remove(out_path)

        lf_data_cancertype.sink_csv(out_path, separator="\t")

    #return lf_data_cancertype.collect()


In [112]:
cancer_type_list = ["PAAD", "SKCM", "LAML", "CHOL", "CESC", "DLBC", "HNSC", "BRCA", "LUAD", "LUSC"]

for c_type in cancer_type_list: 
    make_cancer_specific_tsv(c_type, True)

In [179]:
make_cancer_specific_tsv("PAAD", True)

ComputeError: ComputeError: The output type of the 'apply' function cannot be determined.
The function was never called because 'skip_nulls=True' and all values are null.
Consider setting 'skip_nulls=False' or setting the 'return_dtype'.

In [142]:
df_genenames["converted_alias"][1]

# Add a column of gene_ids: 

all_columns = df_c_type.columns
df = df_c_type.with_columns(pl.col("transcript_id").map_elements(fetch_gene_ids_from_ensembl).alias("gene_id")).select([pl.col("transcript_id"), pl.col("gene_id")]  + [pl.col(c) for c in all_columns if c not in ["gene_id", "transcript_id"]] ) 


In [181]:
cancer_type_abb = "PAAD"

df_pheno = pl.read_csv(in_pheno2, separator="\t")
sample_id_cancertype = df_pheno.filter(pl.col("cancer type abbreviation") == cancer_type_abb).select(pl.col("sample")).to_numpy()
#sample_id_data = np.array(lf_data.collect_schema().names())
sample_id_data = np.array(lf_data.limit(1).collect().columns)
sample_id_cancertype = set(np.intersect1d(sample_id_cancertype, sample_id_data))

# Subset lazyframe to include only samples from cancer type of interest.
lf_data_cancertype = lf_data.select(sample_id_cancertype.union(set(["sample"])))

# Use list to filter the data: 
lf_data_cancertype = lf_data_cancertype.filter(pl.col("sample").is_in(trans_id_of_interest))


In [185]:
lf_data_cancertype.with_columns(pl.col("sample").map_elements(fetch_gene_ids_from_ensembl).alias("gene_id")).collect()

TCGA-M8-A5N4-01,TCGA-IB-7890-01,TCGA-Q3-AA2A-01,TCGA-LB-A9Q5-01,TCGA-IB-8126-01,TCGA-3A-A9IZ-01,TCGA-HZ-7924-01,TCGA-HZ-A49H-01,TCGA-IB-A5SO-01,TCGA-2J-AABU-01,TCGA-IB-A6UG-01,TCGA-3A-A9IC-01,TCGA-IB-7887-01,TCGA-H6-A45N-11,TCGA-HV-A5A3-01,TCGA-3E-AAAY-01,TCGA-IB-AAUO-01,TCGA-3A-A9IL-01,TCGA-H6-8124-01,TCGA-HZ-8315-01,TCGA-2J-AABR-01,TCGA-FB-AAPZ-01,TCGA-HZ-A8P0-01,TCGA-F2-6880-01,TCGA-HV-A7OP-01,TCGA-H6-8124-11,TCGA-HZ-A4BK-01,TCGA-XD-AAUH-01,TCGA-3A-A9IS-01,TCGA-RB-A7B8-01,TCGA-IB-A5SP-01,TCGA-IB-AAUT-01,TCGA-IB-7644-01,TCGA-HZ-A8P1-01,TCGA-3A-A9IR-01,TCGA-2J-AABK-01,TCGA-XN-A8T5-01,…,TCGA-3A-A9IJ-01,TCGA-S4-A8RM-01,TCGA-HZ-7919-01,TCGA-IB-A5ST-01,TCGA-IB-A5SS-01,TCGA-F2-A44H-01,TCGA-YH-A8SY-01,TCGA-F2-A44G-01,TCGA-2J-AABO-01,TCGA-HZ-7925-01,TCGA-3A-A9IN-01,TCGA-FB-AAPQ-01,TCGA-3A-A9I5-01,TCGA-2J-AABH-01,TCGA-3A-A9IX-01,TCGA-3E-AAAZ-01,TCGA-FB-AAQ0-01,TCGA-HZ-8005-01,TCGA-IB-7646-01,TCGA-YB-A89D-01,TCGA-IB-AAUU-01,TCGA-LB-A7SX-01,TCGA-HZ-8001-01,TCGA-HZ-7920-01,TCGA-PZ-A5RE-01,TCGA-2J-AAB1-01,TCGA-HV-AA8X-01,TCGA-IB-A7M4-01,TCGA-F2-6879-01,TCGA-IB-7647-01,TCGA-RB-AA9M-01,TCGA-FB-A7DR-01,TCGA-FB-AAQ6-01,TCGA-F2-7273-01,TCGA-IB-AAUV-01,TCGA-FB-AAQ3-01,gene_id
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-0.5125,-2.5479,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-6.5064,-2.1779,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-3.1714,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,…,-9.9658,-9.9658,-2.8262,-3.0469,-1.2481,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-0.5543,-1.0862,-9.9658,-9.9658,-9.9658,-9.9658,-2.3884,-3.1714,-9.9658,-2.2447,-9.9658,-9.9658,-1.7809,-9.9658,-3.1714,-9.9658,-9.9658,-9.9658,-2.9324,"""ARG2-202"""
-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-1.9942,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,…,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.3147,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-1.9942,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,"""LGALS3-202"""
-9.9658,-3.3076,-3.458,-9.9658,-1.9379,-9.9658,-2.9324,-4.2934,-1.9379,-2.3147,-4.035,-9.9658,-9.9658,-1.5951,-9.9658,-4.035,-0.9686,-3.816,-2.2447,-2.4659,-3.458,-3.458,-4.035,-9.9658,-9.9658,-2.9324,-2.7274,-3.816,-9.9658,-9.9658,-9.9658,-3.458,-9.9658,-9.9658,-9.9658,-3.6259,-9.9658,…,-9.9658,-2.1779,-2.5479,-9.9658,-9.9658,-9.9658,-3.816,-3.3076,-9.9658,-4.2934,-9.9658,-2.8262,-9.9658,-3.6259,-3.1714,-9.9658,-9.9658,-2.5479,-3.3076,-2.5479,-2.8262,-9.9658,-9.9658,-2.114,-2.1779,-9.9658,-9.9658,-9.9658,-4.2934,-3.3076,-3.816,-3.6259,-9.9658,-2.8262,-9.9658,-9.9658,"""CD274-204"""
-9.9658,-9.9658,-9.9658,-9.9658,-3.1714,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-5.0116,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-4.035,-9.9658,…,-3.6259,-9.9658,-2.8262,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.114,-2.0529,-4.035,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-5.0116,-3.458,-9.9658,"""CD274-205"""
-5.5735,-5.0116,-3.3076,-9.9658,-5.5735,-5.5735,-3.1714,-9.9658,-6.5064,-9.9658,-5.0116,-5.0116,-5.5735,-5.5735,-5.5735,-4.6082,-4.2

In [162]:
#"sample" in lf_data_PAAD_subset.columns

all_columns = lf_data_PAAD.columns
lf_data_PAAD.with_columns(pl.col("transcript_id").map_elements(fetch_gene_ids_from_ensembl).alias("gene_id")).select([pl.col("transcript_id"), pl.col("gene_id")]  + [pl.col(c) for c in all_columns if c not in ["gene_id", "transcript_id"]] ).collect()

transcript_id,gene_id,TCGA-M8-A5N4-01,TCGA-IB-7890-01,TCGA-Q3-AA2A-01,TCGA-LB-A9Q5-01,TCGA-IB-8126-01,TCGA-3A-A9IZ-01,TCGA-HZ-7924-01,TCGA-HZ-A49H-01,TCGA-IB-A5SO-01,TCGA-2J-AABU-01,TCGA-IB-A6UG-01,TCGA-3A-A9IC-01,TCGA-IB-7887-01,TCGA-H6-A45N-11,TCGA-HV-A5A3-01,TCGA-3E-AAAY-01,TCGA-IB-AAUO-01,TCGA-3A-A9IL-01,TCGA-H6-8124-01,TCGA-HZ-8315-01,TCGA-2J-AABR-01,TCGA-FB-AAPZ-01,TCGA-HZ-A8P0-01,TCGA-F2-6880-01,TCGA-HV-A7OP-01,TCGA-H6-8124-11,TCGA-HZ-A4BK-01,TCGA-XD-AAUH-01,TCGA-3A-A9IS-01,TCGA-RB-A7B8-01,TCGA-IB-A5SP-01,TCGA-IB-AAUT-01,TCGA-IB-7644-01,TCGA-HZ-A8P1-01,TCGA-3A-A9IR-01,…,TCGA-2J-AAB4-01,TCGA-3A-A9IJ-01,TCGA-S4-A8RM-01,TCGA-HZ-7919-01,TCGA-IB-A5ST-01,TCGA-IB-A5SS-01,TCGA-F2-A44H-01,TCGA-YH-A8SY-01,TCGA-F2-A44G-01,TCGA-2J-AABO-01,TCGA-HZ-7925-01,TCGA-3A-A9IN-01,TCGA-FB-AAPQ-01,TCGA-3A-A9I5-01,TCGA-2J-AABH-01,TCGA-3A-A9IX-01,TCGA-3E-AAAZ-01,TCGA-FB-AAQ0-01,TCGA-HZ-8005-01,TCGA-IB-7646-01,TCGA-YB-A89D-01,TCGA-IB-AAUU-01,TCGA-LB-A7SX-01,TCGA-HZ-8001-01,TCGA-HZ-7920-01,TCGA-PZ-A5RE-01,TCGA-2J-AAB1-01,TCGA-HV-AA8X-01,TCGA-IB-A7M4-01,TCGA-F2-6879-01,TCGA-IB-7647-01,TCGA-RB-AA9M-01,TCGA-FB-A7DR-01,TCGA-FB-AAQ6-01,TCGA-F2-7273-01,TCGA-IB-AAUV-01,TCGA-FB-AAQ3-01
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ENST0000055649…","""ARG2-202""",-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-0.5125,-2.5479,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-6.5064,-2.1779,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-3.1714,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,…,-9.9658,-9.9658,-9.9658,-2.8262,-3.0469,-1.2481,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-0.5543,-1.0862,-9.9658,-9.9658,-9.9658,-9.9658,-2.3884,-3.1714,-9.9658,-2.2447,-9.9658,-9.9658,-1.7809,-9.9658,-3.1714,-9.9658,-9.9658,-9.9658,-2.9324
"""ENST0000055349…","""LGALS3-202""",-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-1.9942,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,…,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.3147,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-1.9942,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
"""ENST0000049292…","""CD274-204""",-9.9658,-3.3076,-3.458,-9.9658,-1.9379,-9.9658,-2.9324,-4.2934,-1.9379,-2.3147,-4.035,-9.9658,-9.9658,-1.5951,-9.9658,-4.035,-0.9686,-3.816,-2.2447,-2.4659,-3.458,-3.458,-4.035,-9.9658,-9.9658,-2.9324,-2.7274,-3.816,-9.9658,-9.9658,-9.9658,-3.458,-9.9658,-9.9658,-9.9658,…,-4.035,-9.9658,-2.1779,-2.5479,-9.9658,-9.9658,-9.9658,-3.816,-3.3076,-9.9658,-4.2934,-9.9658,-2.8262,-9.9658,-3.6259,-3.1714,-9.9658,-9.9658,-2.5479,-3.3076,-2.5479,-2.8262,-9.9658,-9.9658,-2.114,-2.1779,-9.9658,-9.9658,-9.9658,-4.2934,-3.3076,-3.816,-3.6259,-9.9658,-2.8262,-9.9658,-9.9658
"""ENST0000049826…","""CD274-205""",-9.9658,-9.9658,-9.9658,-9.9658,-3.1714,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-5.0116,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,…,-4.035,-3.6259,-9.9658,-2.8262,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.114,-2.0529,-4.035,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-5.0116,-3.458,-9.9658
"""ENST0000051258…","""TDO2-207""",-5.5735,-5.0116,-3.3076,-9.9658,-5.5735,-5.5735,-3.1

In [160]:
lf_data_PAAD.select("transcript_id").collect()

transcript_id
str
"""ENST0000055649…"
"""ENST0000055349…"
"""ENST0000049292…"
"""ENST0000049826…"
"""ENST0000051258…"
"""ENST0000022193…"
"""ENST0000025351…"
"""ENST0000052163…"
"""ENST0000026006…"
